In [1]:
## import libraries and modules ##

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import os
import torch

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, TensorBoard

from statsmodels.tsa.statespace.sarimax import SARIMAX

: 

In [3]:
## GPU use ##

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using CPU


In [4]:
## Tensorboard ##
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [5]:
## Access Google Drive ##

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
## Data preprocessing ##

# Load & extract data
data_rain = pd.read_csv('/content/drive/MyDrive/data_mrc/Rainfall_StungTreng.csv')
data_flow = pd.read_csv('/content/drive/MyDrive/data_mrc/Discharge_StungTreng.csv')

data_rain = data_rain.rename(columns={'Value': 'rainfall'})
data_flow = data_flow.rename(columns={'Value': 'flowrate'})

data_rain = data_rain[['Timestamp (UTC+07:00)', 'rainfall']]
data_flow = data_flow[['Timestamp (UTC+07:00)', 'flowrate']]

# Merge data
data = pd.merge(data_rain, data_flow, on='Timestamp (UTC+07:00)', how='inner')

# Normalize data
scaler = MinMaxScaler()
data[['rainfall', 'flowrate']] = scaler.fit_transform(data[['rainfall', 'flowrate']])

# Create sequences
def create_sequences(input_data, target_data, sequence_length):

    xs = []
    ys = []
    for i in range(len(input_data)-sequence_length):
        xs.append(input_data.iloc[i:(i+sequence_length)].values)
        ys.append(target_data.iloc[i+sequence_length])
    return np.array(xs), np.array(ys)

seq_length = 120  # Day series length to consider for prediction
X, y = create_sequences(data[['rainfall']], data['flowrate'], seq_length)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3658, 120, 1), (915, 120, 1), (3658,), (915,))

In [ ]:
## Models ##

model = Sequential([
    Input((seq_length, 1)),
    LSTM(50, activation='tanh'),
    Dense(1)
])
model.compile(optimizer="adam", loss='mean_squared_error', metrics=[tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.MeanAbsoluteError()])

model2 = Sequential([
    Input((seq_length, 1)),
    Bidirectional(LSTM(50, activation='tanh', return_sequences=True)),
    Dropout(0.2),
    LSTM(50, activation='tanh'),
    Dropout(0.2),
    Dense(1)
])

model2.compile(optimizer="adam", loss='mean_squared_error', metrics=[tf.keras.metrics.MeanSquaredError(), tf.keras.metrics.MeanAbsoluteError()])

model3 = SARIMAX(data['flowrate'], order=(1, 1, 1), seasonal_order= (1, 1, 1, 12), exog= data['rainfall'])

In [ ]:
## Training and evaluation ##

# Training
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.1)

In [ ]:
## Model 2 training with early stopping ##
early_stopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

# Learning rate decay function
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * np.exp(-0.1)

#lr_scheduler = LearningRateScheduler(scheduler)
history2 = model2.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.1, callbacks=[early_stopping , tensorboard_callback])

In [ ]:
# Plotting the training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model 1 Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history2.history['loss'], label='Training Loss')
plt.plot(history2.history['val_loss'], label='Validation Loss')
plt.title('Model 2 Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history3.history['loss'], label='Training Loss')
plt.plot(history3.history['val_loss'], label='Validation Loss')
plt.title('Model 3 Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

In [ ]:
# Evaluation
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

loss2 = model2.evaluate(X_test, y_test)
print(f'Test Loss 2: {loss2}')


# Prediction
y_pred = model.predict(X_test)
y_pred2 = model2.predict(X_test)

# Calculate MSE and MAE
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

mse2 = mean_squared_error(y_test, y_pred2)
mae2 = mean_absolute_error(y_test, y_pred2)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'Mean Squared Error 2: {mse2}')
print(f'Mean Absolute Error 2: {mae2}')

In [ ]:
## Plot ##

plt.figure(figsize=(12, 6))
plt.plot(y_test[550:600], label='True')
plt.plot(y_pred[550:600], label='Predicted')
#plt.plot(y_pred2[550:600], label='Predicted 2')
plt.legend()
plt.xlabel('Epochs')
plt.title('Flowrate')

plt.show()

## Save model ##
model.save('model.h5')
model2.save('model2.h5')
#model3.save('model_sarimax.h5')